In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install -r yolov5/requirements.txt
!pip install gdown

Cloning into 'yolov5'...
remote: Enumerating objects: 12208, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 12208 (delta 6), reused 0 (delta 0), pack-reused 12195
Receiving objects: 100% (12208/12208), 11.91 MiB | 14.66 MiB/s, done.
Resolving deltas: 100% (8450/8450), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 4.6 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import gdown

random.seed(108)

In [ ]:
# URL Sequences
# https://drive.google.com/file/d/1s5rLE3Ij9AH5TkzK4Lg6VnmDZln6K9Ck/view?usp=sharing
# URL Frameonly
# https://drive.google.com/file/d/1yw2v7g-UorewE6bTTo-AJSN-RNCups7W/view?usp=sharing
url = 'https://drive.google.com/uc?id=1s5rLE3Ij9AH5TkzK4Lg6VnmDZln6K9Ck'
output = '/content/sequences.zip'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=1yw2v7g-UorewE6bTTo-AJSN-RNCups7W'
output = '/content/frameonly.zip'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1s5rLE3Ij9AH5TkzK4Lg6VnmDZln6K9Ck
To: /content/sequences.zip
100%|██████████| 58.0M/58.0M [00:00<00:00, 199MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yw2v7g-UorewE6bTTo-AJSN-RNCups7W
To: /content/frameonly.zip
100%|██████████| 164M/164M [00:00<00:00, 270MB/s]


'/content/frameonly.zip'

In [ ]:
!unzip sequences.zip -d ''
!unzip frameonly.zip -d ''

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: Sequence/EDD2020_SA144510032.txt  
  inflating: Sequence/EDD2020_SA144510022.txt  
  inflating: Sequence/EDD2020_M144520017.txt  
  inflating: Sequence/EDD2020_SA144510019.txt  
  inflating: Sequence/Pylseg4_000173.txt  
  inflating: Sequence/EDD2020_SA144510039.txt  
  inflating: Sequence/EDD2020_SA144510025.txt  
  inflating: Sequence/EDD2020_SA144510014.txt  
  inflating: Sequence/EDD2020_SA144510043.txt  
  inflating: Sequence/EDD2020_M144520003.txt  
  inflating: Sequence/EDD2020_SA144510024.txt  
  inflating: Sequence/EDD2020_SA144510002.txt  
  inflating: Sequence/EDD2020_SA144510004.txt  
  inflating: Sequence/EDD2020_SA144510023.txt  
  inflating: Sequence/EDD2020_SA144510007.txt  
  inflating: Sequence/EDD2020_SA144510026.txt  
  inflating: Sequence/EDD2020_SA144510020.txt  
  inflating: Sequence/EDD2020_SA144510035.txt  
  inflating: Sequence/EDD2020_SA144510009.txt  
  inflating: Sequence/EDD2020_SA14

In [ ]:
!mkdir dataset dataset/images dataset/annot
!mkdir dataset/images/train dataset/images/val dataset/images/test 
!mkdir dataset/annot/train dataset/annot/val dataset/annot/test

def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

for bigFolder in ['framesOnly', 'Sequence']:
  files = [os.path.join(bigFolder, x) for x in os.listdir(bigFolder)]
  files.sort()
  images = [f for f in files if '.jpg' in f]
  annot = [f for f in files if '.txt' in f]

  if bigFolder == 'framesOnly':
    train_images, val_images, train_annotations, val_annotations = train_test_split(images, annot, test_size = 0.2, random_state = 1)
  else:
    test_images, test_annotations = images, annot


move_files_to_folder(train_images, 'dataset/images/train')
move_files_to_folder(val_images, 'dataset/images/val/')
move_files_to_folder(test_images, 'dataset/images/test/')
move_files_to_folder(train_annotations, 'dataset/annot/train/')
move_files_to_folder(val_annotations, 'dataset/annot/val/')
move_files_to_folder(test_annotations, 'dataset/annot/test/')
!mv dataset/annot dataset/labels

In [ ]:
cd yolov5

/content/yolov5


In [ ]:
with open("class_data.yaml", "w") as data:
  for folder in ['test', 'train', 'val']:
    data.write(f'{folder}: ../dataset/images/{folder}/\n')
  data.write('nc: 8\n')
  data.write('names: ["specularity", "saturation", "artifact", "blur", "contrast", "bubbles", "instrument", "blood"]\n')


In [ ]:
# 125         
# mapAt=0.5
# steps=2
# 161 
#    iouv = torch.linspace(mapAt, 0.75, steps, device=device)  # iou vector for mAP@0.5:0.95
# 188
#     s = ('%20s' + '%11s' * 6) % ('Class', 'Images', 'Labels', 'P', 'R', f'mAP@{mapAt}', f'mAP@{mapAt}:.75')
# 354
#     parser.add_argument('--mapAt', type=float, default=0.5, help='map@{}')
#     parser.add_argument('--steps', type=int, default=2, help='step')



In [ ]:
!python train.py --img 512 --cfg yolov5s.yaml --hyp hyp.scratch-low.yaml --batch 32 --epochs 10 --data class_data.yaml --weights yolov5s.pt --workers 24 --name yolo_ead

train: weights=yolov5s.pt, cfg=yolov5s.yaml, data=class_data.yaml, hyp=hyp.scratch-low.yaml, epochs=10, batch_size=32, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=24, project=runs/train, name=yolo_ead, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-247-gcf298fb Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0

In [ ]:
!python train.py --img 512 --cfg yolov5m.yaml --hyp hyp.scratch-low.yaml --batch 32 --epochs 10 --data class_data.yaml --weights yolov5m.pt --workers 24 --name yolo_ead_m

train: weights=yolov5m.pt, cfg=yolov5m.yaml, data=class_data.yaml, hyp=hyp.scratch-low.yaml, epochs=10, batch_size=32, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=24, project=runs/train, name=yolo_ead_m, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-247-gcf298fb Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v

In [ ]:
!python train.py --img 512 --cfg yolov5x.yaml --hyp hyp.scratch-low.yaml --batch 8 --epochs 10 --data class_data.yaml --weights yolov5x.pt --workers 24 --name yolo_ead_x

train: weights=yolov5x.pt, cfg=yolov5x.yaml, data=class_data.yaml, hyp=hyp.scratch-low.yaml, epochs=10, batch_size=8, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=24, project=runs/train, name=yolo_ead_x, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-247-gcf298fb Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=

In [ ]:
!python train.py --img 512 --cfg yolov5x.yaml --hyp hyp.scratch-med.yaml --batch 8 --epochs 10 --data class_data.yaml --weights yolov5x.pt --workers 24 --name yolo_ead_x_med

train: weights=yolov5x.pt, cfg=yolov5x.yaml, data=class_data.yaml, hyp=hyp.scratch-med.yaml, epochs=10, batch_size=8, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=24, project=runs/train, name=yolo_ead_x_med, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-247-gcf298fb Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.3, cls_pw=1.0, obj=0.7, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv

## Experiments

In [ ]:
!python train.py --img 512 --cfg yolov5s.yaml --hyp hyp.scratch-low.yaml --batch 32 --epochs 100 --data class_data.yaml --weights yolov5s.pt --workers 24 --name yolo_ead_s_long

train: weights=yolov5s.pt, cfg=yolov5s.yaml, data=class_data.yaml, hyp=hyp.scratch-low.yaml, epochs=100, batch_size=32, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=24, project=runs/train, name=yolo_ead_s_long, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-247-gcf298fb Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7,

In [ ]:
!python train.py --img 512 --cfg yolov5m.yaml --hyp hyp.scratch-med.yaml --batch 32 --epochs 100 --data class_data.yaml --weights yolov5m.pt --workers 24 --name yolo_ead_m_long

train: weights=yolov5m.pt, cfg=yolov5m.yaml, data=class_data.yaml, hyp=hyp.scratch-med.yaml, epochs=100, batch_size=32, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=24, project=runs/train, name=yolo_ead_m_long, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-247-gcf298fb Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.3, cls_pw=1.0, obj=0.7, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, 

In [ ]:
!python train.py --img 512 --cfg yolov5x.yaml --hyp hyp.scratch-high.yaml --batch 8 --epochs 100 --data class_data.yaml --weights yolov5x.pt --workers 24 --name yolo_ead_x_long

In [ ]:
!python val.py --weights runs/train/yolo_ead_s_long3/weights/best.pt --data class_data.yaml --task val --name yolo_ead_s_test_25 --mapAt 0.25 --steps 10

In [ ]:
!python val.py --weights runs/train/yolo_ead_s_long3/weights/best.pt --data class_data.yaml --task val --name yolo_ead_s_test_50 --mapAt 0.50

In [ ]:
!python val.py --weights runs/train/yolo_ead_s_long/weights/best.pt --data class_data.yaml --task val --name yolo_ead_s_test_75 --mapAt 0.75

In [ ]:
!python val.py --weights runs/train/yolo_ead_m_long/weights/best.pt --data class_data.yaml --task val --name yolo_ead_m_test_25 --mapAt 0.25 --steps 10

In [ ]:
!python val.py --weights runs/train/yolo_ead_m_long/weights/best.pt --data class_data.yaml --task val --name yolo_ead_m_test_50 --mapAt 0.50

In [ ]:
!python val.py --weights runs/train/yolo_ead_m_long/weights/best.pt --data class_data.yaml --task val --name yolo_ead_m_test_75 --mapAt 0.75

In [ ]:
!python val.py --weights runs/train/yolo_ead_x_long/weights/best.pt --data class_data.yaml --task val --name yolo_ead_x_test_25 --mapAt 0.25 --steps 10

In [ ]:
!python val.py --weights runs/train/yolo_ead_x_long/weights/best.pt --data class_data.yaml --task val --name yolo_ead_x_test_50 --mapAt 0.50

In [ ]:
!python val.py --weights runs/train/yolo_ead_x_long/weights/best.pt --data class_data.yaml --task val --name yolo_ead_x_test_75 --mapAt 0.75